In [48]:
import numpy as np 
import pandas as pd
import random
import os

np.random.seed(45)

In [49]:
no_dup_path='../Our Labeled Data/No_Dup/no_d/'
contains_dup_path='../Our Labeled Data/Contains_Dup/contains_d/'

In [50]:
no_dup_files = os.listdir(no_dup_path)
contains_dup_files = os.listdir(contains_dup_path)

### 1. Approach to create labeled data with features - word1, word2, isDuplicate

1. Randomly pick one file from the contains_duplicate dataset
2. Randomly pick two rows with
3. Populate the output data with words and isDuplicate value according to the entity number from the two pairs of rows


In this approach we completely ignore the data statistics and context information

In [43]:
df1= pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})
for _ in range(100):
    file=random.choice(contains_dup_files)
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    for _ in range(10):
        t=tempdf.sample(n=2).reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
        df1= df1.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)

In [44]:
df1[df1['isDuplicate']==0].shape, df1[df1['isDuplicate']==1].shape

((914, 7), (86, 7))

In [45]:
df1

w1             w2  count1  count2    total1    total2  \
0                y              n  2260.0  2260.0  158026.0  158026.0   
1                N              Y  6295.0  6295.0  158026.0  158026.0   
2                y              N  2260.0  2260.0  158026.0  158026.0   
3                y              n  2260.0  2260.0  158026.0  158026.0   
4                n              y   877.0   877.0  158026.0  158026.0   
..             ...            ...     ...     ...       ...       ...   
995  san francisco  San Francisco    50.0    50.0    5807.0    5807.0   
996  san francisco      New York,    50.0    50.0    5807.0    5807.0   
997       New York  san francisco  1773.0  1773.0    5807.0    5807.0   
998  san francisco      New York,    50.0    50.0    5807.0    5807.0   
999    Chicago, IL        Chicago     1.0     1.0    5807.0    5807.0   

     isDuplicate  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  
..           ...  
995          1.0  
996          0.0  
997          0.0  
998          0.0  
999          1.0  

[1000 rows x 7 columns]

### 2. Approach to create labeled data with features - word1, word2, isDuplicate

1. Randomly pick one file from the contains_duplicate dataset
2. Randomly pick two rows with a. same entitity number, b. different entitity number
3. Populate the output data with words and isDuplicate value according to the entity number from the two pairs of rows


In this approach we completely ignore the data statistics and context information

In [ ]:
df2 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

for _ in range(10000):
    file=random.choice(contains_dup_files)
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    
    perEntityCatCount=tempdf['Entity_Number'].value_counts()

    for _ in range(5):
        tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
        
        try:
            t=tempdf[tempdf['entityCount']>1].sample(n=1).reset_index()
            e=t.loc[0]['Entity_Number']
            t=tempdf[tempdf['Entity_Number']==e].sample(n=2).reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,l2,e2=t.loc[1]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
        except ValueError:
            pass

        try:
#             t=tempdf[tempdf['entityCount']==1].sample(n=2).reset_index()
            t=tempdf.sample(n=2).reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,l2,e2=t.loc[1]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)

        except ValueError:
            pass

In [97]:
df2['isDuplicate'].value_counts()

1.0    3000
0.0    2271
Name: isDuplicate, dtype: int64

In [98]:
df3 = df2.drop_duplicates()

w1  \
0                                              orange   
1                                                pink   
2                                              orange   
3                                                 tan   
4                                             silver    
5                                               sable   
6         Engineering and Engineering Trades Programs   
7   Other Programs in Education at the Third Level...   
8         Engineering and Engineering Trades Programs   
9                              First Year High School   
10        Engineering and Engineering trades Programs   
11             Social and Behavioral Science Programs   
12                                        DURHAM LEAD   
13                                          Glen Park   
14                                        Durham Lead   

                                             w2  isDuplicate  
0                                       orange           1.0  
1                                           tan          0.0  
2                                       orange           1.0  
3                                       apricot          0.0  
4                                        silver          1.0  
5                                          pink          0.0  
6   Engineering and Engineering trades Programs          1.0  
7                    Second Year Post Secondary          0.0  
8   Engineering and Engineering trades Programs          1.0  
9          Business and Administration Programs          0.0  
10  Engineering and Engineering Trades Programs          1.0  
11                                      Grade 4          0.0  
12                                  Durham Lead          1.0  
13                                  Mount Rowan          0.0  
14                                  DURHAM LEAD          1.0

In [39]:
df3['isDuplicate'].value_counts()

0.0    2066
1.0     983
Name: isDuplicate, dtype: int64